In [4]:
import os
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import json
from httplib2 import Http   

In [19]:
def connectService():
    #     scope - using spreadsheets in readonly format
    scopes = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    
    # key file generated by 
    # create credentials and service accounts
    # https://developers.google.com/sheets/quickstart/python
    # https://developers.google.com/api-client-library/python/auth/service-accounts
    # share spreadsheet with the client-email available in key generated in service-accounts
    
    # key generated in service-accounts
    try:
        file_name = '/home/vishal/websites/admission/profileadd/key_file_spreadsheetAPi-137ed6b3cf96.json'
    except:
        print('Json file key missing')

    # get credentials
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        file_name, scopes=scopes)

    # authorize the credentials
    http_auth = credentials.authorize(Http())
    # this is for sheets
    discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                    'version=v4')
    try:
        service = build('sheets', 'v4', http=http_auth,discoveryServiceUrl=discoveryUrl)
    except:
        print('service build problem')
    return service


def connectSpreadsheet(spreadsheetid):
    try:
        service = connectService()
    except:
        print('Error in google service connection, check it')
        return None
    # get properties of sheet
    result = service.spreadsheets().get(
        spreadsheetId=spreadsheetid, fields='sheets.properties').execute()
    return result


def spreadsheetProperties(sheet_id, datas):
    datas = datas['sheets']
    for data in datas:
        if data['properties']['sheetId'] == sheet_id:
            return data
    return None


def spreadsheetDatas():
    # spreadsheet Id
    spreadsheetid = '15-1xegxsfH-f_0poJKKboS-F0l3qT44gpc3lTY3yjXo'
    result = connectSpreadsheet(spreadsheetid)
    if result is not None:
        spreadsheet_properties = spreadsheetProperties(0, result)
    else:
        print('spreadsheet connection problem')
        return None
    
    if spreadsheet_properties is not None:
        print(json.dumps(spreadsheet_properties, sort_keys=False, indent=4))
        #     result fetching for sheet1 with array value 0
        result_fetched = spreadsheet_properties["properties"]['gridProperties']
        #     sheet title
        sheet_title = spreadsheet_properties['properties']['title']
        print(sheet_title)
        # get number of rows and columns
        row_count = result_fetched['rowCount']
        frozen_column_count = result_fetched['frozenColumnCount']
        column_count = result_fetched['columnCount']
        frozen_row_count = result_fetched['frozenRowCount']
        # row_count to taken into action
        actual_row_count = row_count - frozen_row_count
        # range_name = sheet_title+'!'+'A'+str(frozen_row_count)+':'+'P'+str(row_count)

        if actual_row_count is not 0 or None and column_count is not 0 or None:
            print('Non-zero values')
            try:
                service = connectService()
            except:
                print('Error in google service connection, check it')
                return None
            BroadInsightsConditions.objects.all().delete()
            if row_count < 10:
                # Sheet1!A2:P3  
                print('less than 10 ranges')
                row_count_analyze = frozen_row_count+1
                range_name = sheet_title+'!'+'A'+str(row_count_analyze)+':'+'P'+str(row_count)
                print(range_name)
                insert_db(service, spreadsheetId, range_name)
            else:
                print('greater than 10 ranges')
                row_count_analyze = frozen_row_count+1
                row_step = 10
                for i in range(row_count_analyze, row_count, row_step):
                    # Sheet1!A2:P3  
                    range_name = sheet_title+'!'+'A'+str(i)+':'+'P'+str(i+9)
                    print(range_name)
                    insert_in_db(service, spreadsheetid, range_name)
    else:
        print('No data in Spreadsheet')
        
#     rangeName = 'Sheet1!A2:P3'
#     rangeName = range_name
#     result = service.spreadsheets().values().get(
#         spreadsheetId=spreadsheetId, range=rangeName, valueRenderOption= 'UNFORMATTED_VALUE',
#         majorDimension='ROWS').execute()
#     # print(result)
#     print(json.dumps(result, sort_keys=False, indent=4))


def insert_in_db(service, spreadsheetid, range_name):
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheetid, range=range_name,majorDimension='ROWS',
        valueRenderOption= 'FORMATTED_VALUE').execute()
    values = result['values']
    # print(json.dumps(values, sort_keys=False, indent=4))
    for row in values:
        bic = BroadInsightsConditions()
        if len(row) is not 0:
            try:
                bic.rule_id = int(row[0])
                print(type(row[1]))
                bic.sub_rule_id = row[1] 
                print(bic.sub_rule_id)
                bic.subject = row[2]
                bic.versions = row[3]
                bic.priority = int(row[4])
                bic.rule_title = row[5]
                bic.derived = int(row[6])
                bic.derived_from = row[7]
                bic.dt_14764 = int(row[8])
                bic.mt_25196 = int(row[9])
                bic.mt_26007 = int(row[10])
                bic.mt_27905 = int(row[11])
                bic.mt_28065 = int(row[12])
                bic.condition_data = row[13]
                bic.insight = row[14]
                bic.save()
                print('saved data '+str(bic.sub_rule_id))
            except Exception as e:
                bic.save()
                print(str(e) +"Saved Data with Exception "+row[1])
        
    

In [20]:
spreadsheetDatas()

{
    "properties": {
        "title": "Sheet1",
        "sheetId": 0,
        "index": 0,
        "gridProperties": {
            "frozenRowCount": 2,
            "columnCount": 17,
            "rowCount": 25,
            "frozenColumnCount": 2
        },
        "sheetType": "GRID"
    }
}
Sheet1
Non-zero values
greater than 10 ranges
Sheet1!A3:P12
<class 'str'>
1_1
saved data 1_1
<class 'str'>
1_2
saved data 1_2
<class 'str'>
1_3
saved data 1_3
<class 'str'>
1_4
saved data 1_4
<class 'str'>
1_5
list index out of rangeSaved Data with Exception 1_5
<class 'str'>
1_6
list index out of rangeSaved Data with Exception 1_6
<class 'str'>
1_7
list index out of rangeSaved Data with Exception 1_7
<class 'str'>
1_8
invalid literal for int() with base 10: ''Saved Data with Exception 1_8
<class 'str'>
1_9
list index out of rangeSaved Data with Exception 1_9
<class 'str'>
1_10
list index out of rangeSaved Data with Exception 1_10
Sheet1!A13:P22
<class 'str'>
1_11
list index out of rangeSaved Data 

In [2]:
from django.core.cache import cache

In [3]:
cache.set("test",'Got it', timeout=None)
a_dict = {'key':'va', 'ke':'val', 'ked':'fla'}
a_dict1 = {'key':'va', 'ke':'val', 'ked':'fla'}
a_dict2 = {'key':'va', 'ke':'val', 'ked':'fla'}
a_dict3 = {'key':'va', 'ke':'val', 'ked':'fla'}

cache.set('a_dict', a_dict)
cache.set('a_dict1', a_dict1)
cache.set('a_dict2', a_dict2)
cache.set('a_dict3', a_dict3)

True

In [4]:
cache.ttl("test")

In [5]:
cache.expire("a_dict", timeout=5)

In [6]:
cache.keys('a_*')

['a_dict2', 'a_dict1', 'a_dict3', 'a_dict']

In [7]:
next(cache.iter_keys('a_*'))

'a_dict2'

In [8]:
cache.delete_pattern('a_*')

3

In [9]:
import pprint
import json
BroadInsightsConditions.objects.all()
list(BroadInsightsConditions.objects.values_list())
json.dumps(list(BroadInsightsConditions.objects.values()), sort_keys=False, indent=4)
BroadInsightsConditions.objects.values()

[{'mt_27905': 1, 'sub_rule_id': '1_1', 'condition_data': '1', 'priority': 1, 'mt_25196': 1, 'mt_26007': 1, 'rule_title': 'x', 'mt_28065': 1, 'derived_from': '0', 'dt_14764': 1, 'insight': '{\n    "data": {\n      "time_consider_last": 310,\n      "no_questions": 12,\n      "no_ques_consider_wrong": 2,\n      "time_spent": 540,\n      "no_ques_consider_low": 2,\n      "time_class_low": "High",\n      "status": "Rght"\n    },\n    "visual": {\n      "status": "Rght"\n    },\n  }', 'versions': 'v2', 'rule_id': 1, 'subject': 'verbal', 'derived': 0, 'id': 70}, {'mt_27905': 1, 'sub_rule_id': '1_2', 'condition_data': '1', 'priority': 1, 'mt_25196': 1, 'mt_26007': 1, 'rule_title': 'y', 'mt_28065': 1, 'derived_from': '0', 'dt_14764': 1, 'insight': '{\n    "data": {\n      "time_consider_last": 310,\n      "no_questions": 13,\n      "no_ques_consider_wrong": 2,\n      "time_spent": 540,\n      "no_ques_consider_low": 2,\n      "time_class_low": "High",\n      "status": "Rght"\n    },\n    "visua

In [10]:
for values in BroadInsightsConditions.objects.values():
    print(values.get('rule_id'))
    cache.set(values.get('rule_id'), values)
    

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
3
3
4
4
5


In [5]:
def make_key(key, key_prefix, version):
    return ':'.join([str(key), str(key_prefix), str(version)])

In [6]:
make_key('1','1-1','v2')

'1:1-1:v2'

In [7]:
cache.set(make_key('1','1-1','v1'),values)
cache.set(make_key('1','1-1','v2'),values)
cache.set(make_key('1','1-1','v3'),values)

True

In [8]:
cache.get(make_key('1','1-1','v1'))
cache.get(make_key('1','1-1','v2'))
cache.get(make_key('1','1-1','v3'))

{'condition_data': '',
 'derived': None,
 'derived_from': '',
 'dt_14764': None,
 'id': 92,
 'insight': '',
 'mt_25196': None,
 'mt_26007': None,
 'mt_27905': None,
 'mt_28065': None,
 'priority': None,
 'rule_id': 5,
 'rule_title': '',
 'sub_rule_id': '4_3',
 'subject': '',
 'versions': ''}

In [9]:
cache.keys('1:1-1:*')

['1:1-1:v2', '1:1-1:v1', '1:1-1:v3']

In [11]:
cache.get(make_key('1','1-1','v3'))

{'condition_data': '',
 'derived': None,
 'derived_from': '',
 'dt_14764': None,
 'id': 92,
 'insight': '',
 'mt_25196': None,
 'mt_26007': None,
 'mt_27905': None,
 'mt_28065': None,
 'priority': None,
 'rule_id': 5,
 'rule_title': '',
 'sub_rule_id': '4_3',
 'subject': '',
 'versions': ''}

In [11]:
cache.scan('1:1-1:*')

AttributeError: 'RedisCache' object has no attribute 'scan'

In [7]:
def connectService():
    #     scope - using spreadsheets in readonly format
    scopes = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    
    # key file generated by 
    # create credentials and service accounts
    # https://developers.google.com/sheets/quickstart/python
    # https://developers.google.com/api-client-library/python/auth/service-accounts
    # share spreadsheet with the client-email available in key generated in service-accounts
    
    # key generated in service-accounts
    try:
        file_name = '/home/vishal/websites/admission/profileadd/key_file_spreadsheetAPi-137ed6b3cf96.json'
    except:
        print('Json file key missing')

    # get credentials
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        file_name, scopes=scopes)

    # authorize the credentials
    http_auth = credentials.authorize(Http())
    # this is for sheets
    discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                    'version=v4')
    try:
        service = build('sheets', 'v4', http=http_auth,discoveryServiceUrl=discoveryUrl)
    except:
        print('service build problem')
    return service


def connectSpreadsheet(spreadsheetid):
    try:
        service = connectService()
    except:
        print('Error in google service connection, check it')
        return None
    # get properties of sheet
    result = service.spreadsheets().get(
        spreadsheetId=spreadsheetid, fields='sheets.properties').execute()
    return result


def spreadsheetProperties(sheet_id, datas):
    datas = datas['sheets']
    for data in datas:
        if data['properties']['sheetId'] == sheet_id:
            return data
    return None


def spreadsheetDatas():
    # spreadsheet Id
    spreadsheetid = '15-1xegxsfH-f_0poJKKboS-F0l3qT44gpc3lTY3yjXo'
    print(spreadsheetid)
    result = connectSpreadsheet(spreadsheetid)
    if result is not None:
        spreadsheet_properties = spreadsheetProperties(0, result)
    else:
        print('spreadsheet connection problem')
        return None
    
    if spreadsheet_properties is not None:
        print(json.dumps(spreadsheet_properties, sort_keys=False, indent=4))
        #     result fetching for sheet1 with array value 0
        result_fetched = spreadsheet_properties["properties"]['gridProperties']
        #     sheet title
        sheet_title = spreadsheet_properties['properties']['title']
        print(sheet_title)
        # get number of rows and columns
        row_count = result_fetched['rowCount']
        frozen_column_count = result_fetched['frozenColumnCount']
        column_count = result_fetched['columnCount']
        frozen_row_count = result_fetched['frozenRowCount']
        # row_count to taken into action
        actual_row_count = row_count - frozen_row_count
        # range_name = sheet_title+'!'+'A'+str(frozen_row_count)+':'+'P'+str(row_count)

        if actual_row_count is not 0 or None and column_count is not 0 or None:
            print('Non-zero values')
            try:
                service = connectService()
            except:
                print('Error in google service connection, check it')
                return None
            delete_all_data()
            if row_count < 10:
                # Sheet1!A2:P3  
                print('less than 10 ranges')
                row_count_analyze = frozen_row_count+1
                range_name = sheet_title+'!'+'A'+str(row_count_analyze)+':'+'P'+str(row_count)
                print(range_name)
                insert_db(service, spreadsheetId, range_name)
            else:
                print('greater than 10 ranges')
                row_count_analyze = frozen_row_count+1
                row_step = 10
                for i in range(row_count_analyze, row_count, row_step):
                    # Sheet1!A2:P3  
                    range_name = sheet_title+'!'+'A'+str(i)+':'+'P'+str(i+9)
                    print(range_name)
                    values = spreadsheet_cell_data(service, spreadsheetid, range_name)
                    insert_all_data(values)
    else:
        print('No data in Spreadsheet')
        
#     rangeName = 'Sheet1!A2:P3'
#     rangeName = range_name
#     result = service.spreadsheets().values().get(
#         spreadsheetId=spreadsheetId, range=rangeName, valueRenderOption= 'UNFORMATTED_VALUE',
#         majorDimension='ROWS').execute()
#     # print(result)
#     print(json.dumps(result, sort_keys=False, indent=4))

def delete_all_data():
    print('Delete all Objects in BroadInsightsConditions')
    BroadInsightsConditions.objects.all().delete()

def spreadsheet_cell_data(service, spreadsheetid, range_name):
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheetid, range=range_name,majorDimension='ROWS',
        valueRenderOption= 'FORMATTED_VALUE').execute()
    values = result['values']
    return values


def insert_all_data(values):
    # print(json.dumps(values, sort_keys=False, indent=4))
    for row in values:
        bic = BroadInsightsConditions()
        if len(row) is not 0:
            try:
                
                bic.unique_id = str(row[0])
                print(bic.unique_id)
                bic.rule_id = int(row[1])
                bic.sub_rule_id = row[2] 
                bic.subject = row[3]
                bic.versions = row[4]
                bic.priority = int(row[5])
                bic.rule_title = row[6]
                bic.derived = int(row[7])
                bic.derived_from = row[8]
                bic.dt_14764 = int(row[9])
                bic.mt_25196 = int(row[10])
                bic.mt_26007 = int(row[11])
                bic.mt_27905 = int(row[12])
                bic.mt_28065 = int(row[13])
                bic.condition_data = row[14]
                bic.insight = row[15]
                bic.save()
                print('saved data '+str(bic.sub_rule_id))
            except Exception as e:
                bic.save()
                print(str(e) +"Saved Data with Exception "+row[1])
        
    

In [20]:
service = connectService()
spreadsheetid = '15-1xegxsfH-f_0poJKKboS-F0l3qT44gpc3lTY3yjXo'

In [19]:
result = connectSpreadsheet(spreadsheetid)
spreadsheet_properties = spreadsheetProperties(0, result)
result_fetched = spreadsheet_properties["properties"]['gridProperties']
sheet_title = spreadsheet_properties['properties']['title']
row_count = result_fetched['rowCount']

In [21]:
row_count_analyze = frozen_row_count+1
row_step = 10
for i in range(row_count_analyze, row_count, row_step):
    # Sheet1!A2:P3  
    range_name = sheet_title+'!'+'A'+str(i)+':'+'P'+str(i+9)
    print(range_name)
    values = spreadsheet_cell_data(service, spreadsheetid, range_name)
    insert_all_data(values)

Sheet1!A3:P12
1-1_1-v2
saved data 1_1
1-1_2-v1
saved data 1_2
1-1_3-v1
saved data 1_3
1-1_4-v1
saved data 1_4
1-1_5-v2
list index out of rangeSaved Data with Exception 1
1-1_6-v3
list index out of rangeSaved Data with Exception 1
1-1_7-v4
list index out of rangeSaved Data with Exception 1
1-1_8-v5
invalid literal for int() with base 10: ''Saved Data with Exception 1
1-1_9-v6
list index out of rangeSaved Data with Exception 1
1-1_10-v7
list index out of rangeSaved Data with Exception 1
Sheet1!A13:P22
1-1_11-v8
list index out of rangeSaved Data with Exception 1
1-1_12-v9
list index out of rangeSaved Data with Exception 1
1-1_13-v10
list index out of rangeSaved Data with Exception 1
1-1_14-v11
list index out of rangeSaved Data with Exception 1
1-1_15-v12
list index out of rangeSaved Data with Exception 1
2-2_1-v13
list index out of rangeSaved Data with Exception 2
2-2_2-v14
list index out of rangeSaved Data with Exception 2
2-2_3-v15
list index out of rangeSaved Data with Exception 2
3-3_

In [ ]:
spreadsheet_cell_data(service,spreadsheetid,'Sheet1!A2:Q2' )